In [ ]:
!pip install azure-ai-formrecognizer

In [ ]:
# Training label included model (Already labeled on )
import os
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.core.credentials import AzureKeyCredential

# Set user key, endpoint
key = "653310d794cow5fks674b91918a7dc72" # Your Form Recognizer API key
endpoint = "https://xxxxxxxxxx.cognitiveservices.azure.com/" # Your Form Recognizer API endpoint

# Call API
form_recognizer_client = FormRecognizerClient(endpoint, AzureKeyCredential(key))
form_training_client = FormTrainingClient(endpoint, AzureKeyCredential(key))

# To train a model you need an Azure Storage account.
# Use the SAS URL to access your training files. (the directly blob's SAS URL, not whole storage)
trainingDataUrl = "https://xxxxxxxx.blob.core.windows.net/xxxxxxxxxxxxxx"

poller = form_training_client.begin_training(trainingDataUrl, use_training_labels=True)
model = poller.result()
trained_model_id = model.model_id

print("Model ID: {}".format(trained_model_id))
print("Status: {}".format(model.status))
print("Training started on: {}".format(model.training_started_on))
print("Training completed on: {}".format(model.training_completed_on))

print("\nRecognized fields:")
for submodel in model.submodels:
    print(
        "The submodel with form type '{}' has recognized the following fields: {}".format(
            submodel.form_type,
            ", ".join(
                [
                    field.label if field.label else name
                    for name, field in submodel.fields.items()
                ]
            ),
        )
    )

# Training result information
for doc in model.training_documents:
    print("Document name: {}".format(doc.name))
    print("Document status: {}".format(doc.status))
    print("Document page count: {}".format(doc.page_count))
    print("Document errors: {}".format(doc.errors))

In [ ]:
# by url
import os
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.core.credentials import AzureKeyCredential

# Set user key, endpoint
key = "653310d794cow5fks674b91918a7dc72" # Your Form Recognizer API key
endpoint = "https://xxxxxxxxxx.cognitiveservices.azure.com/" # Your Form Recognizer API endpoint

# call API
form_recognizer_client = FormRecognizerClient(endpoint, AzureKeyCredential(key))
trained_model_id = "ae18dd01-e1af-44dc-b378-a849c1c4ec4d"


# URL
formUrl = "https://s.zimedia.com.tw/s/G5YEQz-3"

poller = form_recognizer_client.begin_recognize_custom_forms_from_url(
    model_id=trained_model_id, form_url=formUrl)
result = poller.result()

output = {}

for recognized_form in result:
    print("Form type: {}".format(recognized_form.form_type))
#     print(recognized_form.fields)
    for name, field in recognized_form.fields.items():
        if name not in output:
            output[name]= str(field.value)
        else:
            output[name].append(field.value)

output["年份"] = output["年份"].replace(' 年','')
output["月份"] = output["月份"].replace(' 月','')
output["發票號碼"] = output["發票號碼"][-8:]
print(output)

In [ ]:
# by local
import os
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.core.credentials import AzureKeyCredential

# Set user key, endpoint
key = "653310d794cow5fks674b91918a7dc72" # Your Form Recognizer API key
endpoint = "https://xxxxxxxxxx.cognitiveservices.azure.com/" # Your Form Recognizer API endpoint

# call API
form_recognizer_client = FormRecognizerClient(endpoint, AzureKeyCredential(key))
trained_model_id = "ae18dd01-e1af-44dc-b378-a849c1c4ec4d"

# image location
local_image_path = os.getcwd() + '/photo/test/test__42385416.jpg'

# 讀取圖片 (二進位)
local_image = open(local_image_path, "rb")

poller = form_recognizer_client.begin_recognize_custom_forms(model_id=trained_model_id, form=local_image)
result = poller.result()

output = {}

for recognized_form in result:
    print("Form type: {}".format(recognized_form.form_type))
#     print(recognized_form.fields)
    for name, field in recognized_form.fields.items():
        if name not in output:
            output[name]= str(field.value)
        else:
            output[name].append(field.value)

output["年份"] = output["年份"].replace(' 年','')
output["月份"] = output["月份"].replace(' 月','')
output["發票號碼"] = output["發票號碼"][-8:]
print(output)